In [19]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

In [20]:
data = pd.read_csv("emoji_data.csv", header=None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [21]:
emoji_dictionary = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

In [22]:
X = data[0].values
Y = data[1].values

Y

array([4, 3, 3, 1, 2, 1, 4, 3, 4, 1, 3, 3, 2, 2, 4, 3, 2, 3, 3, 1, 3, 2,
       2, 2, 0, 1, 0, 4, 2, 0, 2, 0, 0, 3, 4, 0, 2, 1, 3, 1, 0, 4, 0, 3,
       0, 4, 2, 3, 4, 2, 2, 3, 0, 2, 2, 3, 2, 3, 2, 2, 3, 3, 0, 2, 3, 0,
       2, 0, 0, 2, 3, 2, 4, 1, 3, 3, 0, 0, 3, 2, 0, 3, 0, 2, 2, 4, 2, 2,
       0, 0, 2, 3, 0, 4, 2, 1, 2, 3, 3, 2, 3, 0, 3, 0, 2, 0, 2, 3, 4, 3,
       1, 3, 4, 3, 2, 3, 3, 3, 1, 4, 4, 2, 2, 1, 1, 2, 3, 2, 3, 4, 2, 3,
       0, 2, 0, 0, 4, 3, 4, 2, 3, 2, 3, 4, 2, 1, 2, 4, 3, 1, 3, 2, 3, 2,
       2, 3, 3, 2, 4, 0, 0, 0, 3, 0, 0, 1, 1, 2, 2, 2, 0, 3, 2, 3, 3, 1,
       2, 2, 4, 2, 3, 1, 2], dtype=int64)

Embeddings

In [23]:
with open('data/glove_dataset/glove.6B.100d.txt','r', encoding='utf8') as file:
    content = file.readlines()

In [24]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype=float)

convert input text into tokens

In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word_to_index = tokenizer.word_index


In [26]:
Xtokens = tokenizer.texts_to_sequences(X)


In [27]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    
    return maxlen
maxlen = get_maxlen(Xtokens)

In [28]:
Xtrain = pad_sequences(Xtokens, maxlen = maxlen, padding = 'post', truncating = 'post')

In [29]:
Ytrain = to_categorical(Y)

Model

In [30]:
embed_size = 100

In [31]:
embedding_matrix = np.zeros((len(word_to_index)+1, embed_size))

for word, i in word_to_index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [32]:
model = Sequential([
    Embedding(input_dim = len(word_to_index)+1,
              output_dim = embed_size,
              input_length=maxlen,
              weights = [embedding_matrix],
              trainable = False
              ),
    LSTM(units = 16, return_sequences=True),
    LSTM(units = 4),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(Xtrain, Ytrain, epochs=100)

Epoch 1/100
6/6 [==============================] - 3s 6ms/step - loss: 1.6020 - accuracy: 0.2514
Epoch 2/100
6/6 [==============================] - 0s 10ms/step - loss: 1.5757 - accuracy: 0.3169
Epoch 3/100
6/6 [==============================] - 0s 7ms/step - loss: 1.5570 - accuracy: 0.3279
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 1.5415 - accuracy: 0.3279
Epoch 5/100
6/6 [==============================] - 0s 8ms/step - loss: 1.5269 - accuracy: 0.3443
Epoch 6/100
6/6 [==============================] - 0s 7ms/step - loss: 1.5150 - accuracy: 0.3934
Epoch 7/100
6/6 [==============================] - 0s 8ms/step - loss: 1.4992 - accuracy: 0.4372
Epoch 8/100
6/6 [==============================] - 0s 10ms/step - loss: 1.4818 - accuracy: 0.4590
Epoch 9/100
6/6 [==============================] - 0s 9ms/step - loss: 1.4604 - accuracy: 0.4863
Epoch 10/100
6/6 [==============================] - 0s 12ms/step - loss: 1.4363 - accuracy: 0.4863
Epoch 11/100
6/6 [========

In [34]:
test = ["I am trying", "I want to cry", "This is just sad"]